In [2]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/sample_submission.csv
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/train.csv
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/test.csv
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4515480.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4517890.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4517724.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4518419.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_7767967.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4537730.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4637585.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_4535651.jpg
/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/img_7764741.

## Loading required packages

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from keras.models import Model
from keras.optimizers import Adam
import cv2
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.optimizers import Adam

## Loading training data

In [5]:
df = pd.read_csv('../input/fast-furious-and-insured/Fast_Furious_Insured/train.csv')
df.head()

,Image_path,Insurance_company,Cost_of_vehicle,Min_coverage,Expiry_date,Max_coverage,Condition,Amount
0,img_4513976.jpg,BQ,41500.0,1037.5,2026-12-03,36142.68,0,0.0
1,img_7764995.jpg,BQ,50700.0,1267.5,2025-07-10,12753.00,1,6194.0
2,img_451308.jpg,A,49500.0,1237.5,2022-08-11,43102.68,0,0.0
3,img_7768372.jpg,A,33500.0,837.5,2022-08-02,8453.00,1,7699.0
4,img_7765274.jpg,AC,27600.0,690.0,2026-05-01,6978.00,1,8849.0


## Loading testing data

In [4]:
df_test = pd.read_csv('../input/fast-furious-and-insured/Fast_Furious_Insured/test.csv')
df_test

,Image_path,Insurance_company,Cost_of_vehicle,Min_coverage,Expiry_date,Max_coverage
0,img_4538519.jpg,B,23600,590.0,2025-04-12,5978.00
1,img_7766002.jpg,C,28300,707.5,2028-08-24,7153.00
2,img_4637390.jpg,AC,43700,1092.5,2023-11-28,11003.00
3,img_4516108.jpg,BB,46100,1152.5,2028-02-04,11603.00
4,img_4517008.jpg,BB,40700,1017.5,2022-01-03,10253.00
...,...,...,...,...,...,...
595,img_7766518.jpg,B,30900,772.5,2024-10-23,7803.00
596,img_4535713.jpg,O,51300,1282.5,2025-02-21,12903.00
597,img_4511787.jpg,BQ,27000,675.0,2023-07-13,23527.68
598,img_4517592.jpg,AA,42600,1065.0,2024-05-05,10728.00


## Pre-processing

In [6]:
# Get the number of missing data points per column
missing_values_count_train = df.isnull().sum()
print(missing_values_count_train)

Image_path            0
Insurance_company     0
Cost_of_vehicle      89
Min_coverage         89
Expiry_date           0
Max_coverage         89
Condition             0
Amount               11
dtype: int64


In [7]:
# Get the number of missing data points per column
missing_values_count_test = df_test.isnull().sum()
print(missing_values_count_test)

Image_path           0
Insurance_company    0
Cost_of_vehicle      0
Min_coverage         0
Expiry_date          0
Max_coverage         0
dtype: int64


In [10]:
# Filling missing values
df = df.fillna(method='bfill', axis=0).fillna(0)

In [9]:
# Checking different values in Insurance company in the training set
df['Insurance_company'].value_counts()

B     148
A     132
C     131
BQ    131
DA    128
O     126
BB    124
AA    123
BC    121
AC    119
RE    116
Name: Insurance_company, dtype: int64

In [12]:
# Checking different values in Insurance company in the testing set
df_test['Insurance_company'].value_counts()

BB    67
AA    62
A     60
C     60
B     59
RE    54
BQ    52
AC    50
O     50
BC    43
DA    43
Name: Insurance_company, dtype: int64

# Label encoding and scaling

In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from keras.models import Model
from keras.optimizers import Adam
import cv2
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.optimizers import Adam
features_num = ['Cost_of_vehicle', 'Min_coverage', 'Max_coverage']
features_cat = ['Insurance_company']

le= LabelEncoder()   
df['Insurance_company'] = le.fit_transform(df['Insurance_company'])
df_test['Insurance_company'] = le.transform(df_test['Insurance_company'])

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
)

y = df['Amount']
train_imputed = df.loc[:,['Cost_of_vehicle', 'Min_coverage', 'Max_coverage', 'Insurance_company']]
X = preprocessor.fit_transform(train_imputed)

test_imputed = df_test.loc[:,['Cost_of_vehicle', 'Min_coverage',  'Max_coverage', 'Insurance_company']]
test_X = preprocessor.transform(test_imputed)

train_imputed.columns

Index(['Cost_of_vehicle', 'Min_coverage', 'Max_coverage', 'Insurance_company'], dtype='object')

In [16]:
#Train-test split
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1,test_size=0.2)

## Train a random forest regressor

In [17]:
rf_model = RandomForestRegressor(random_state=1, n_estimators = 1000, max_depth=3)
# fit your model
rf_model.fit(train_X,train_y)
val_preds = rf_model.predict(val_X)
# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(val_y,val_preds)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 2050.061640924192


## Get the predictions for amount

In [19]:
amount_predictions = rf_model.predict(test_X)

## Prepare the images

In [20]:
X = df.loc[:,['Image_path']]
y = df.loc[:,['Condition']]    
X_test = df.loc[:,['Image_path']]
print('train set shape:', X.shape)
print('test set shape:', X_test.shape)

train set shape: (1399, 1)
test set shape: (1399, 1)


In [24]:
data = []
labels = []
for (index_label, row_series) in df.iterrows():
        img_path = row_series.values[0]
        condition = row_series.values[-2]
        labels.append(int(condition))
        # load the image, pre-process it, and store it in the data list
        originalImage = cv2.imread('/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/trainImages/' + img_path)
        image = cv2.resize(originalImage, (224, 224))
        image = img_to_array(image)
        data.append(image)

## Transfer learning with MobileNet

In [25]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)
   # we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:80]:
    layer.trainable=False
for layer in model.layers[80:]:
    layer.trainable=True

In [26]:
from tensorflow.keras import optimizers

print(len(data),len(labels))
data = np.array(data, dtype="float")
labels = np.array(labels)
    
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(train_images, test_images, train_labels, test_labels) = train_test_split(data,labels, test_size=0.2, random_state=42)

#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0)-0.5, (test_images / 255.0) -0.5

train_labels = to_categorical(train_labels, 2)
test_labels = to_categorical(test_labels, 2)

#compile and train the model
adam=optimizers.Adam(
                lr=0.002,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=None,
                decay=0.0001,
                amsgrad=False
                )

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#callback = callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_images, train_labels, batch_size=32,epochs=5,shuffle=True, validation_data=(test_images, test_labels))

1399 1399
Epoch 1/5
35/35 [==============================] - 8s 73ms/step - loss: 0.4563 - accuracy: 0.8972 - val_loss: 0.1823 - val_accuracy: 0.9571
Epoch 2/5
35/35 [==============================] - 1s 39ms/step - loss: 0.0796 - accuracy: 0.9727 - val_loss: 0.1783 - val_accuracy: 0.9500
Epoch 3/5
35/35 [==============================] - 1s 40ms/step - loss: 0.0312 - accuracy: 0.9930 - val_loss: 0.2173 - val_accuracy: 0.9536
Epoch 4/5
35/35 [==============================] - 1s 41ms/step - loss: 0.0134 - accuracy: 0.9960 - val_loss: 0.3882 - val_accuracy: 0.9607
Epoch 5/5
35/35 [==============================] - 1s 39ms/step - loss: 0.0106 - accuracy: 0.9968 - val_loss: 0.2730 - val_accuracy: 0.9536


## Getting the test prediction

In [27]:
condition_predictions = []
for (index_label, row_series) in df_test.iterrows():
        img_path = row_series.values[0]
        # load the image, pre-process it, and store it in the data list
        originalImage = cv2.imread('/kaggle/input/fast-furious-and-insured/Fast_Furious_Insured/testImages/' + img_path)
        image = cv2.resize(originalImage, (224, 224))
        image = img_to_array(image)
        image = image.reshape((1,224, 224, 3))
        image = np.array(image, dtype="float") / 255.0 - 0.5
        prediction = model.predict(image)
        prediction = prediction[0]
        condition_predictions.append(np.argmax(prediction))
       

## Preparing the submission

In [28]:
submission = pd.DataFrame({'Image_path': df_test.Image_path, 'Condition': condition_predictions, 
                          'Amount': amount_predictions})
submission.to_csv('submission.csv', index=False)
from IPython.display import FileLink
FileLink('submission.csv')


/kaggle/working/submission.csv